# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


In [235]:
X_train.shape

(8000, 20)

## Пишем свой класс для SVM

In [245]:
import numpy as np
from random import randint
import random
import math
np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        a=self.w
        b=np.array([self.w0])
        c= np.hstack((a,b ))
        d=np.hstack((x,[1]))
        ret=np.dot(c, d)
        #print(ret, c)
        return ret

    # a(x) = [M(x) > 0]
   
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        der=self.w/self.C
        '''done ToDo: fix this function'''
        return der

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        if self.loss(x, answer) > 0:
            return -np.dot(x.T, answer)
        return 0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 / math.sqrt(k+1)

            # w update
            self.w-=step*(self.der_loss(X_train[rand_index],  y_train[rand_index])+self.der_reg())
            #self.w-=step*self.der_loss(X_train[rand_index],  y_train[rand_index])
            
            '''done: ToDo: add w update with regularization'''
            
            

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [246]:
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[-0.629852   -0.32367627  0.02432447  1.19340314 -0.73944938  0.32239867
 -0.80184796  0.53560415 -0.76134059 -1.08557651 -0.10890644  0.28654643
  1.19131481  0.33786879 -0.4180304   0.22841889  0.22822097  0.65795351
 -0.07578072  0.14554457] -1.0128311203344238


In [247]:
predictions = model.predict(X_test)

In [248]:
print(predictions)

[0 1 1 ... 1 1 1]


In [249]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 1] 2000 991


In [250]:
print(len(predictions), sum(predictions))

2000 1207


In [251]:
print(sum(predictions == y_test) / float(len(y_test)))

0.73


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

## from sklearn.svm import LinearSVC

In [53]:
modellib=LinearSVC()

In [55]:
modellib.fit(X_train, y_train)
#print(modellib)

LinearSVC()


/home/ivan/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [56]:
predictionslib = modellib.predict(X_test)

In [57]:
print(predictionslib)

[1 0 1 ... 1 1 0]


In [58]:
print(len(predictionslib), sum(predictionslib))

2000 1015


In [59]:
print(sum(predictionslib == y_test) / float(len(y_test)))

0.801
